# Missing Data

In [1]:
# magic to help out Jupyter notebooks
import os, sys
%cd -q ..
sys.path.append(os.path.abspath('src'))

In [2]:
# allow log messages in notebooks
import sys
import logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [3]:
import pandas as pd 
import numpy as np

# use the pecarn module to bring the PECARN dataset into the notebook
from data import pecarn
pecarn_tbi = pecarn.load(fromCsv=False)
data = pecarn.clean(pecarn_tbi)

INFO:data.pecarn.load:Loading from Pickle file c:\Jan\Capstone\PECARN_TBI.pkl
INFO:data.pecarn.clean:Filling missing GCSEye, GCSVerbal, GCSMotor when GCSTotal is 15
INFO:data.pecarn.clean:PosIntFinal Dropping 20 rows where PosIntFinal is NaN
INFO:data.pecarn.clean:Gender Dropping 3 rows where Gender is NaN


In [12]:
pecarn_tbi['VomitNbr'].cat.categories = ['blah','de','blah2','bleh']


In [4]:
pecarn_tbi_nans = pecarn_tbi.isna().sum().sum()
data_nans = data.isna().sum().sum()
fixed_nans = pecarn_tbi_nans - data_nans
percent_fixed_nans = round(fixed_nans / pecarn_tbi_nans * 100, 1)
print(f"{fixed_nans} out of {pecarn_tbi_nans} ({percent_fixed_nans}%) of NaN values in the PECARN TBI dataset have been cleaned")

7497 out of 65584 (11.4%) of NaN values in the PECARN TBI dataset have been cleaned


# Columns with many NaNs
The following table shows the NaN counts in the original PECARN TBI dataset

In [5]:
pecarn_tbi_nan_cols = list(pecarn_tbi.columns[pecarn_tbi.isna().sum() > 0])
data_nan_cols = list(data.columns[data.isna().sum() > 0])

nan_df = pd.DataFrame(columns=['# NaN', '% NaN', 'Fixed NaN', '% Remaining'], index=pecarn_tbi_nan_cols)
nan_df['# NaN'] = pecarn_tbi[pecarn_tbi_nan_cols].isna().sum()
nan_df['% NaN'] = round(nan_df['# NaN'] / pecarn_tbi.shape[0] * 100, 1)

for col in pecarn_tbi_nan_cols:
    if col in data.columns:
        nan_df.loc[col, 'Fixed NaN'] = pecarn_tbi[col].isna().sum() - data[col].isna().sum()
        nan_df.loc[col, '% Remaining'] = round(data[col].isna().sum() / pecarn_tbi.shape[0] * 100, 1)

nan_df = nan_df.sort_values(by='% Remaining', ascending=False)

display(nan_df)

,# NaN,% NaN,Fixed NaN,% Remaining
Ethnicity,15966,36.8,12,36.8
Dizzy,15972,36.8,10,36.8
Race,3208,7.4,1,7.4
Drugs,1818,4.2,1,4.2
HAStart,1332,3.1,0,3.1
VomitLast,992,2.3,1,2.3
NeuroD,660,1.5,1,1.5
Vomit,443,1.0,1,1
SFxBas,445,1.0,1,1
VomitStart,413,1.0,0,1


# Columns with multiple categories
These columns have multi level categories (i.e. not binary) that have NaNs.

We can potentially create a new category representing "Missing" or "Unknown".

In [6]:
# find the column names where there are more than 2 categories and there are NaNs
multi_category_cols_with_nans = [
    col
    for col in pecarn_tbi.select_dtypes(include='category').columns
    if len(pecarn_tbi[col].dtype.categories) > 2 & pecarn_tbi[col].isna().sum() > 0
]
multi_category_cols_with_nans

['CTSed',
 'EmplType',
 'GCSMotor',
 'GCSVerbal',
 'HemaLoc',
 'HemaSize',
 'High_impact_InjSev',
 'SFxPalpDepress',
 'SeizOccur']

In [7]:
pecarn_tbi.select_dtypes(include='category')['AMS'].dtype.categories


KeyError: 'AMS'

# Rows with many NaNs
Some rows have a large number of NaN values, these could be nuisance records.

In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pecarn_tbi[(pecarn_tbi.isna().sum(axis=1) > 18)]

,AMS,AMSAgitated,AMSOth,AMSRepeat,AMSSleep,AMSSlow,ActNorm,AgeInMonth,AgeTwoPlus,AgeinYears,Amnesia_verb,CTDone,CTForm1,CTSed,CTSedAge,CTSedAgitate,CTSedOth,CTSedRqst,Certification,Clav,ClavFace,ClavFro,ClavNeck,ClavOcc,ClavPar,ClavTem,DeathTBI,Dizzy,Drugs,EDCT,EDDisposition,EmplType,Ethnicity,Finding1,Finding10,Finding11,Finding12,Finding13,Finding14,Finding2,Finding20,Finding21,Finding22,Finding23,Finding3,Finding4,Finding5,Finding6,Finding7,Finding8,Finding9,FontBulg,GCSEye,GCSGroup,GCSMotor,GCSTotal,GCSVerbal,Gender,HASeverity,HAStart,HA_verb,Hema,HemaLoc,HemaSize,High_impact_InjSev,HospHead,HospHeadPosCT,IndAMS,IndAge,IndAmnesia,IndClinSFx,IndHA,IndHema,IndLOC,IndMech,IndNeuroD,IndOth,IndRqstMD,IndRqstParent,IndRqstTrauma,IndSeiz,IndVomit,IndXraySFx,InjuryMech,Intub24Head,Intubated,LOCSeparate,LocLen,NeuroD,NeuroDCranial,NeuroDMotor,NeuroDOth,NeuroDReflex,NeuroDSensory,Neurosurgery,OSI,OSIAbdomen,OSICspine,OSICut,OSIExtremity,OSIFlank,OSIOth,OSIPelvis,Observed,Paralyzed,PosCT,PosIntFinal,Race,SFxBas,SFxBasHem,SFxBasOto,SFxBasPer,SFxBasRet,SFxBasRhi,SFxPalp,SFxPalpDepress,Sedated,Seiz,SeizLen,SeizOccur,Vomit,VomitLast,VomitNbr,VomitStart
PatNum,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11930,<NA>,92,92,92,92,92,<NA>,75,2,6,NaN,False,<NA>,92,92,92,92,92,2,<NA>,92,92,92,92,92,92,False,<NA>,<NA>,92,1,5,NaN,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,False,NaN,2,NaN,15,NaN,1,92,92,NaN,<NA>,92,92,1,False,False,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,6,False,False,0,92,<NA>,92,92,92,92,92,False,<NA>,92,92,92,92,92,92,92,<NA>,False,92,False,2,<NA>,92,92,92,92,92,NaN,92,False,False,92,92,<NA>,92,92,92
26576,<NA>,92,92,92,92,92,True,193,2,16,NaN,True,True,0,92,92,92,92,3,<NA>,92,92,92,92,92,92,False,<NA>,<NA>,1,3,5,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,NaN,2,NaN,15,NaN,1,NaN,NaN,1,<NA>,92,92,3,False,False,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,2,False,<NA>,1,NaN,<NA>,92,92,92,92,92,False,<NA>,92,92,92,92,92,92,92,False,<NA>,0,False,1,<NA>,92,92,92,92,92,NaN,92,<NA>,False,92,92,False,92,92,92
29196,<NA>,92,92,92,92,92,True,21,1,1,91,False,False,92,92,92,92,92,3,<NA>,92,92,92,92,92,92,False,<NA>,<NA>,92,1,3,NaN,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,<NA>,NaN,2,NaN,15,NaN,1,92,92,91,<NA>,92,92,2,False,False,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,8,False,<NA>,0,92,<NA>,92,92,92,92,92,False,<NA>,92,92,92,92,92,92,92,<NA>,<NA>,92,False,90,<NA>,92,92,92,92,92,NaN,92,<NA>,False,92,92,<NA>,92,92,92
34488,<NA>,92,92,92,92,92,<NA>,125,2,10,1,False,False,92,92,92,92,92,3,<NA>,92,92,92,92,92,92,False,<NA>,<NA>,92,1,3,2,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,False,NaN,2,NaN,15,NaN,1,92,92,NaN,<NA>,92,92,1,False,False,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,92,6,False,<NA>,2,2,<NA>,92,92,92,92,92,False,<NA>,92,92,92,92,92,92,92,<NA>,<NA>,92,False,1,<NA>,92,92,92,92,92,NaN,92,<NA>,False,92,92,<NA>,92,92,92


In [9]:
data[(data.isna().sum(axis=1) > 15)]

,AMS,AMSAgitated,AMSOther,AMSRepeat,AMSSleep,AMSSlow,ActingNormal,AgeInMonth,AgeTwoPlus,Amnesia,ClavicalTrauma,ClavicalTraumaFace,ClavicalTraumaScalpFrontal,ClavicalTraumaNeck,ClavicalTraumaScalpOccipital,ClavicalTraumaScalpParietal,ClavicalTraumaScalpTemporal,Dizzy,Drugs,Ethnicity,FontBulg,GCSEye,GCSMotor,GCSVerbal,Gender,HeadAcheSeverity,HAStart,HeadAche,Hematoma,HematomaLocation,HematomaSize,InjuryMechanism,Intubated,LossOfConsciousness,LossOfConsciousnessDuration,NeuroD,NeuroDCranial,NeuroDMotor,NeuroDOth,NeuroDReflex,NeuroDSensory,OSI,OSIAbdomen,OSICspine,OSICut,OSIExtremity,OSIFlank,OSIOth,OSIPelvis,Paralyzed,PosIntFinal,Race,SFxBas,SFxBasHem,SFxBasOto,SFxBasPer,SFxBasRet,SFxBasRhi,SFxPalp,SFxPalpDepress,Sedated,Seizure,SeizureLength,SeizureOccurence,Vomit,VomitLast,VomitNumber,VomitStart
PatNum,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5542,False,NA,NA,NA,NA,NA,<NA>,31,2,Pre/Non Verbal,<NA>,NA,NA,NA,NA,NA,NA,<NA>,<NA>,NaN,False,4,6,5,Male,NA,NA,NaN,<NA>,NA,NA,Other,<NA>,No,NA,<NA>,NA,NA,NA,NA,NA,<NA>,NA,NA,NA,NA,NA,NA,NA,<NA>,False,NaN,<NA>,NA,NA,NA,NA,NA,NaN,NA,<NA>,<NA>,NA,NA,<NA>,NA,NA,NA
26576,<NA>,NA,NA,NA,NA,NA,True,193,2,NaN,<NA>,NA,NA,NA,NA,NA,NA,<NA>,<NA>,NaN,False,4,6,5,Male,NaN,NaN,Yes,<NA>,NA,NA,Pedestrian Vehicle,<NA>,Yes,NaN,<NA>,NA,NA,NA,NA,NA,<NA>,NA,NA,NA,NA,NA,NA,NA,<NA>,False,White,<NA>,NA,NA,NA,NA,NA,NaN,NA,<NA>,False,NA,NA,False,NA,NA,NA
28100,<NA>,NA,NA,NA,NA,NA,<NA>,178,2,Yes,<NA>,NA,NA,NA,NA,NA,NA,<NA>,False,Non-Hispanic,False,4,6,5,Male,NA,NA,NaN,<NA>,NA,NA,Other Transport,<NA>,NaN,NA,<NA>,NA,NA,NA,NA,NA,<NA>,NA,NA,NA,NA,NA,NA,NA,<NA>,False,White,<NA>,NA,NA,NA,NA,NA,NaN,NA,<NA>,<NA>,NA,NA,<NA>,NA,NA,NA
32498,True,No,No,No,No,No,<NA>,83,2,NaN,False,NA,NA,NA,NA,NA,NA,<NA>,<NA>,NaN,False,NaN,NaN,NaN,Female,NA,NA,NaN,False,NA,NA,Fall to Ground,True,Yes,NaN,<NA>,NA,NA,NA,NA,NA,False,NA,NA,NA,NA,NA,NA,NA,True,False,NaN,False,NA,NA,NA,NA,NA,No,NA,<NA>,True,NaN,NaN,<NA>,NA,NA,NA
40731,<NA>,NA,NA,NA,NA,NA,<NA>,6,1,Pre/Non Verbal,<NA>,NA,NA,NA,NA,NA,NA,<NA>,False,NaN,<NA>,4,6,5,Male,NA,NA,NaN,<NA>,NA,NA,Object Struck Head,<NA>,NaN,NA,<NA>,NA,NA,NA,NA,NA,<NA>,NA,NA,NA,NA,NA,NA,NA,<NA>,False,White,<NA>,NA,NA,NA,NA,NA,NaN,NA,<NA>,<NA>,NA,NA,<NA>,NA,NA,NA
41886,True,No,No,No,No,No,False,42,2,NaN,<NA>,NA,NA,NA,NA,NA,NA,<NA>,<NA>,NaN,False,3,6,5,Male,NaN,NaN,Yes,<NA>,NA,NA,Sports,<NA>,No,NA,<NA>,NA,NA,NA,NA,NA,<NA>,NA,NA,NA,NA,NA,NA,NA,<NA>,False,Black,<NA>,NA,NA,NA,NA,NA,NaN,NA,<NA>,<NA>,NA,NA,<NA>,NA,NA,NA
